In [ ]:
import pandas as pd
import requests
import urllib
import json
from urllib.parse import urlencode
import json
import pandas as pd

In [ ]:
def dataset_txt_file_download(link, file_name):
  base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
  public_key = link
  final_url = base_url + urlencode(dict(public_key=public_key))
  response = requests.get(final_url)
  download_url = response.json()['href']
  download_response = requests.get(download_url)

  with open(file_name, 'wb') as f:
    f.write(download_response.content)

  df = pd.read_csv(file_name, sep="\t", header = None)
  return df

# Brightkite

## Downloading

In [ ]:
brightkite_total_checkin_df = dataset_txt_file_download('https://disk.yandex.ru/d/eZtwKKyB1NI9oQ', 'Brightkite_totalCheckins.txt')

In [ ]:
brightkite_total_checkin_df.columns = ['user', 'check-in time', 'latitude', 'longitude', 'location_id']
brightkite_total_checkin_df

,user,check-in time,latitude,longitude,location_id
0,0,2010-10-17T01:48:53Z,39.747652,-104.992510,88c46bf20db295831bd2d1718ad7e6f5
1,0,2010-10-16T06:02:04Z,39.891383,-105.070814,7a0f88982aa015062b95e3b4843f9ca2
2,0,2010-10-16T03:48:54Z,39.891077,-105.068532,dd7cd3d264c2d063832db506fba8bf79
3,0,2010-10-14T18:25:51Z,39.750469,-104.999073,9848afcc62e500a01cf6fbf24b797732f8963683
4,0,2010-10-14T00:21:47Z,39.752713,-104.996337,2ef143e12038c870038df53e0478cefc
...,...,...,...,...,...
4747282,58222,2009-01-23T02:30:34Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747283,58224,2009-01-03T15:06:54Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747284,58225,2009-01-20T13:58:14Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0
4747285,58226,2009-01-20T13:30:09Z,33.833333,35.833333,9f6b83bca22411dd85460384f67fcdb0


In [ ]:
brightkite_edges_df = dataset_txt_file_download('https://disk.yandex.ru/i/LOYwqXxn13LBug', 'Brightkite_edges.txt')

In [ ]:
brightkite_edges_df.columns = ['1st friend', '2nd friend']
brightkite_edges_df

,1st friend,2nd friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
428151,58225,58226
428152,58225,58227
428153,58226,58220
428154,58226,58225


## Analytics

Число юзеров, которые чек-инились

In [ ]:
len(brightkite_total_checkin_df.user.unique())

51406

Максимальное количество чек-инов у одного юзера - 2100

Минимальное количество чек-инов - 1

In [ ]:
brightkite_total_checkin_df.user.value_counts()

user
0        2100
13175    2100
13202    2100
13199    2100
659      2100
         ... 
39336       1
39337       1
39341       1
39343       1
58227       1
Name: count, Length: 51406, dtype: int64

Количество Nan в каждой колонке

In [ ]:
brightkite_total_checkin_df[brightkite_total_checkin_df.columns[brightkite_total_checkin_df.isnull().any()]].isnull().sum()

check-in time    6
latitude         6
longitude        6
location_id      6
dtype: int64

Процент Nan  в каждой колонке

In [ ]:
brightkite_total_checkin_df[brightkite_total_checkin_df.columns[brightkite_total_checkin_df.isnull().any()]].isnull().sum() * 100 / brightkite_total_checkin_df.shape[0]

check-in time    0.000126
latitude         0.000126
longitude        0.000126
location_id      0.000126
dtype: float64

Нет пустых ячеек и строк

In [ ]:
brightkite_total_checkin_df[brightkite_total_checkin_df.isna().any(axis=1)]

,user,check-in time,latitude,longitude,location_id
1909858,7573,NaN,NaN,NaN,NaN
1909859,7573,NaN,NaN,NaN,NaN
1909960,7573,NaN,NaN,NaN,NaN
1909961,7573,NaN,NaN,NaN,NaN
3745451,22819,NaN,NaN,NaN,NaN
3745452,22819,NaN,NaN,NaN,NaN


In [ ]:
brightkite_total_checkin_df.dtypes

user               int64
check-in time     object
latitude         float64
longitude        float64
location_id       object
dtype: object

В один момент времени - максимум 14 событий

In [ ]:
brightkite_total_checkin_df['check-in time'].value_counts()

check-in time
2010-09-30T11:03:54Z    14
2010-09-30T11:03:52Z    12
2010-09-30T11:03:51Z    11
2010-09-30T11:03:53Z    10
2010-07-08T14:10:13Z     9
                        ..
2009-12-03T13:45:45Z     1
2009-12-02T14:47:09Z     1
2009-12-02T14:41:04Z     1
2009-12-02T14:28:14Z     1
2009-01-21T00:24:35Z     1
Name: count, Length: 4506822, dtype: int64

In [ ]:
from dateutil.parser import parse

dates = pd.DataFrame()

dates['event date'] = brightkite_total_checkin_df['check-in time']
dates = dates.dropna()

dates['event date'] = dates['event date'].transform(lambda x: parse(x).date())

В один день максимум 10206 событий

In [ ]:
dates['event date'].value_counts()

event date
2008-10-22    10206
2009-03-14     9577
2008-10-21     9418
2009-03-13     9396
2009-07-25     9334
              ...  
2008-04-08        5
2008-03-27        4
2008-03-31        4
2008-04-04        3
2008-03-21        1
Name: count, Length: 942, dtype: int64

Всего 942 дней, 2008 - 2009 годы

In [ ]:
len(dates['event date'].unique())

942

In [ ]:
dates['event date'].unique()

array([datetime.date(2010, 10, 17), datetime.date(2010, 10, 16),
       datetime.date(2010, 10, 14), datetime.date(2010, 10, 13),
       datetime.date(2010, 10, 12), datetime.date(2010, 10, 11),
       datetime.date(2010, 10, 9), datetime.date(2010, 10, 8),
       datetime.date(2010, 10, 7), datetime.date(2010, 10, 6),
       datetime.date(2010, 10, 5), datetime.date(2010, 10, 4),
       datetime.date(2010, 10, 3), datetime.date(2010, 10, 2),
       datetime.date(2010, 10, 1), datetime.date(2010, 9, 30),
       datetime.date(2010, 9, 29), datetime.date(2010, 9, 28),
       datetime.date(2010, 9, 27), datetime.date(2010, 9, 25),
       datetime.date(2010, 9, 24), datetime.date(2010, 9, 23),
       datetime.date(2010, 9, 22), datetime.date(2010, 9, 21),
       datetime.date(2010, 9, 20), datetime.date(2010, 9, 19),
       datetime.date(2010, 9, 18), datetime.date(2010, 9, 16),
       datetime.date(2010, 9, 15), datetime.date(2010, 9, 14),
       datetime.date(2010, 9, 13), datetime.date(

In [ ]:
brightkite_total_checkin_df.longitude.unique()

array([-104.99251 , -105.070814, -105.068532, ...,   11.918092,
         35.577222,   35.578156])

In [ ]:
len(brightkite_total_checkin_df.longitude.unique())

659658

In [ ]:
brightkite_total_checkin_df.latitude.unique()

array([39.747652, 39.891383, 39.891077, ..., 57.685848, 33.853611,
       33.855255])

In [ ]:
len(brightkite_total_checkin_df.latitude.unique())

651008

In [ ]:
brightkite_total_checkin_df.location_id.value_counts()

location_id
00000000000000000000000000000000            254619
ee81ef22a22411ddb5e97f082c799f59             17396
ede07eeea22411dda0ef53e233ec57ca             16896
ee8b1d0ea22411ddb074dbd65f1665cf             16687
ee78cc1ca22411dd9b3d576115a846a7             14487
                                             ...  
ec0a9d02a22411ddacd517d9006173e4                 1
cfa07e2b6e48d5818585dfabaf43c55d82369630         1
94aad8d47b9011dd8e030030487eb504                 1
aa7f27827d0211dda7da0030487eb504                 1
6ba679dce99c11dd8e2f003048c10834                 1
Name: count, Length: 772966, dtype: int64

Максимум 1134 связи у одного пользователя

In [ ]:
brightkite_edges_df['1st friend'].value_counts()

1st friend
40       1134
44       1055
108       854
116       838
159       833
         ... 
7877        1
38315       1
38314       1
38313       1
58227       1
Name: count, Length: 58228, dtype: int64

In [ ]:
brightkite_edges_df['2nd friend'].value_counts()

2nd friend
40       1134
44       1055
108       854
116       838
159       833
         ... 
38676       1
38674       1
38673       1
38672       1
58227       1
Name: count, Length: 58228, dtype: int64

# Gowalla

## Downloading

In [ ]:
gowalla_total_checkin_df = dataset_txt_file_download('https://disk.yandex.ru/d/KVm7gIom6YooWA', 'Gowalla_totalCheckins.txt')

In [ ]:
gowalla_total_checkin_df.columns = ['user', 'check-in time', 'latitude', 'longitude', 'location_id']
gowalla_total_checkin_df

,user,check-in time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878
...,...,...,...,...,...
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724


In [ ]:
gowalla_edges_df = dataset_txt_file_download('https://disk.yandex.ru/d/Sr3FpAa7WG4GdA', 'Gowalla_edges.txt')

In [ ]:
gowalla_edges_df.columns = ['1st friend', '2nd friend']
gowalla_edges_df

,1st friend,2nd friend
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
1900649,196586,196539
1900650,196587,196540
1900651,196588,196540
1900652,196589,196547


## Analytics

Число юзеров, которые чек-инились

In [ ]:
len(gowalla_total_checkin_df.user.unique())

107092

Максимальное количество чек-инов у одного юзера - 2175

Минимальное количество чек-инов - 1

In [ ]:
gowalla_total_checkin_df.user.value_counts()

10971     2175
776       2175
18931     2150
49918     2125
620       2125
          ... 
146467       1
93418        1
93385        1
93288        1
82641        1
Name: user, Length: 107092, dtype: int64

Количество Nan в каждой колонке

In [ ]:
gowalla_total_checkin_df[gowalla_total_checkin_df.columns[gowalla_total_checkin_df.isnull().any()]].isnull().sum()

Series([], dtype: float64)

Процент Nan  в каждой колонке

In [ ]:
gowalla_total_checkin_df[gowalla_total_checkin_df.columns[gowalla_total_checkin_df.isnull().any()]].isnull().sum() * 100 / gowalla_total_checkin_df.shape[0]

Series([], dtype: float64)

Нет пустых ячеек и строк

In [ ]:
gowalla_total_checkin_df[gowalla_total_checkin_df.isna().any(axis=1)]

,user,check-in time,latitude,longitude,location_id


In [ ]:
gowalla_total_checkin_df.dtypes

user               int64
check-in time     object
latitude         float64
longitude        float64
location_id        int64
dtype: object

В один момент времени - максимум 8 событий

In [ ]:
gowalla_total_checkin_df['check-in time'].value_counts()

2010-10-08T17:50:58Z    8
2010-10-02T12:59:14Z    7
2010-08-27T19:01:13Z    7
2010-09-21T23:07:32Z    7
2010-04-10T01:23:01Z    7
                       ..
2010-10-05T12:48:05Z    1
2010-10-05T12:51:12Z    1
2010-10-05T13:15:19Z    1
2010-10-05T13:38:58Z    1
2010-10-08T21:01:49Z    1
Name: check-in time, Length: 5561957, dtype: int64

In [ ]:
from dateutil.parser import parse

dates = pd.DataFrame()

dates['event date'] = gowalla_total_checkin_df['check-in time']
dates = dates.dropna()

dates['event date'] = dates['event date'].transform(lambda x: parse(x).date())

В один день - максимум 51318 событий

In [ ]:
dates['event date'].value_counts()

2010-10-16    51318
2010-10-09    44896
2010-10-15    44122
2010-10-02    42828
2010-10-17    40960
              ...  
2009-02-23        2
2009-03-06        2
2009-02-04        1
2009-02-05        1
2009-02-18        1
Name: event date, Length: 597, dtype: int64

Всего 597 дней, 2009 - 2010 годы

In [ ]:
len(dates['event date'].unique())

597

In [ ]:
dates['event date'].unique()

array([datetime.date(2010, 10, 19), datetime.date(2010, 10, 18),
       datetime.date(2010, 10, 17), datetime.date(2010, 10, 16),
       datetime.date(2010, 10, 12), datetime.date(2010, 10, 11),
       datetime.date(2010, 10, 10), datetime.date(2010, 10, 9),
       datetime.date(2010, 10, 8), datetime.date(2010, 10, 7),
       datetime.date(2010, 10, 6), datetime.date(2010, 10, 5),
       datetime.date(2010, 10, 4), datetime.date(2010, 10, 3),
       datetime.date(2010, 10, 2), datetime.date(2010, 10, 1),
       datetime.date(2010, 9, 30), datetime.date(2010, 9, 29),
       datetime.date(2010, 9, 28), datetime.date(2010, 9, 27),
       datetime.date(2010, 9, 26), datetime.date(2010, 9, 25),
       datetime.date(2010, 9, 24), datetime.date(2010, 9, 22),
       datetime.date(2010, 9, 21), datetime.date(2010, 9, 17),
       datetime.date(2010, 9, 15), datetime.date(2010, 9, 13),
       datetime.date(2010, 9, 12), datetime.date(2010, 9, 11),
       datetime.date(2010, 9, 9), datetime.date(

In [ ]:
gowalla_total_checkin_df.longitude.unique()

array([-9.77951396e+01, -9.77493954e+01, -9.77633858e+01, ...,
       -6.60989588e-01, -5.04577160e-02,  8.78509820e+00])

In [ ]:
len(gowalla_total_checkin_df.longitude.unique())

1246818

In [ ]:
gowalla_total_checkin_df.latitude.unique()

array([30.23590912, 30.26910295, 30.25573099, ..., 51.9172078 ,
       51.71210979, 50.02781153])

In [ ]:
len(gowalla_total_checkin_df.latitude.unique())

1247095

In [ ]:
gowalla_total_checkin_df.location_id.value_counts()

55033      5811
19542      5662
9410       4713
10259      4083
58725      3476
           ... 
1444440       1
1550807       1
2590535       1
1400413       1
4555073       1
Name: location_id, Length: 1280969, dtype: int64

Максимум 14730 связей у одного юзера, хотя бы одну связь имеют 196591 юзер

In [ ]:
gowalla_edges_df['1st friend'].value_counts()

307       14730
220       10185
505        5420
1149       3986
459        3880
          ...  
157020        1
157021        1
157022        1
157023        1
196590        1
Name: 1st friend, Length: 196591, dtype: int64

In [ ]:
gowalla_edges_df['2nd friend'].value_counts()

307       14730
220       10185
505        5420
1149       3986
459        3880
          ...  
155011        1
84699         1
155013        1
155014        1
196590        1
Name: 2nd friend, Length: 196591, dtype: int64